In [ ]:
# Paths to data files and output file on drive
train_data_file = 'train_data.json'
val_data_file = 'valid_data.json' #needs to be modified for test file
pred_out_file = 'prediction_out.json'

In [ ]:
# Import all dependencies
import numpy as np
import pandas as pd
import spacy
import json
# from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

nlp = spacy.load("en_core_web_sm")
# stopwords = nlp.Defaults.stop_words

from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")


# model = TFPegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
# tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

In [ ]:
df_train_init = pd.read_json(train_data_file)
df_val_init = pd.read_json(val_data_file)

display(df_val_init.head(5))

In [ ]:
# df_train_clean = PreprocessData(df_train_init)
# df_val_clean = PreprocessData(df_val_init)

In [ ]:
ARTICLE = df_val_init['argument'].iloc[5]
# print(ARTICLE)
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="tf", max_length=512,truncation = True)
outputs = model.generate(inputs, max_length=80, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0]))
print(df_val_init['conclusion'].iloc[5])

In [ ]:
# ARTICLE = df_val_init['argument'].iloc[2]
# print(ARTICLE)

# print(df_val_init['conclusion'].iloc[5])
gen_conc = str(tokenizer.decode(outputs[0]))
gen_conc = gen_conc.replace("<pad>","").lstrip().rstrip()


In [ ]:
def conclusion_generate(argument):
    
    inputs = tokenizer.encode("summarize: " + argument, return_tensors = "tf", max_length = 512, truncation = True)
    outputs = model.generate(inputs, max_length=150, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    gen_conclusion = str(tokenizer.decode(outputs[0]))
    gen_conclusion = gen_conclusion.replace("<pad>","").lstrip().rstrip()
    
    return gen_conclusion

In [ ]:
df_val_init['generated_conc'] = df_val_init['argument'].apply(lambda x: conclusion_generate(x))

In [ ]:
# # Contraction mapping dictionary obtained from internet

# contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

#                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

#                            "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

#                            "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

#                            "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

#                            "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

#                            "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

#                            "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

#                            "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

#                            "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

#                            "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

#                            "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

#                            "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

#                            "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

#                            "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

#                            "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

#                            "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

#                            "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

#                            "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

#                            "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

#                            "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

#                            "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

#                            "you're": "you are", "you've": "you have"}


In [ ]:
# # Function to Preprocess the text data: Lower Casing, Remove URLs, punctuations and stripping extra spaces
# def PreprocessData(df):
#   for column in ['conclusion','argument']:
#       df['clean_' + column] = df[column].str.lower()
#       df['clean_' + column] = df['clean_' + column].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ') #remove URL
#       df['clean_' + column] = df['clean_' + column].apply(lambda x: ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in x.split()]))
#       df['clean_' + column] = df['clean_' + column].str.strip()
#       df['clean_' + column] = df['clean_' + column].str.replace('[^\w\s]','')
#       df['clean_' + column] = df['clean_' + column].str.replace('\n','')
#       df['clean_' + column] = df['clean_' + column].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
#   return df

# df['response'] = df['response'].apply(lambda x: [item for item in x.split() if item not in stop])

In [ ]:
# # Argument tokenizer

# # Fit the tokenzier on train data
# arg_tokenizer = Tokenizer(oov_token=1)
# arg_tokenizer.fit_on_texts(df_train_clean['clean_argument'].values)

# # Transform the texts to integer sequences - train and val data
# x_train_arg = arg_tokenizer.texts_to_sequences(df_train_clean['clean_argument'].values)
# x_val_arg = arg_tokenizer.texts_to_sequences(df_val_clean['clean_argument'].values)

# # Pad length for uniformity
# max_length = max(len(s.split()) for s in df_train_clean['clean_argument'].values)
# x_train_arg = pad_sequences(x_train_arg,maxlen=max_length)
# x_val_arg = pad_sequences(x_val_arg,maxlen=max_length)

# # Conclusion tokenizer

# # Fit the tokenzier on train data
# conc_tokenizer = Tokenizer(oov_token=1)
# conc_tokenizer.fit_on_texts(df_train_clean['clean_conclusion'].values)

# # Transform the texts to integer sequences - train and val data
# x_train_conc = conc_tokenizer.texts_to_sequences(df_train_clean['clean_conclusion'].values)
# x_val_conc = conc_tokenizer.texts_to_sequences(df_val_clean['clean_conclusion'].values)

# # Pad length for uniformity
# max_length = max(len(s.split()) for s in df_train_clean['clean_conclusion'].values)
# x_train_conc = pad_sequences(x_train_conc,maxlen=max_length)
# x_val_conc = pad_sequences(x_val_conc,maxlen=max_length)

In [ ]:
# ARTICLE_TO_SUMMARIZE = (
# "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
# "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
# "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
# )
# article = df_val_init['argument'].iloc[1]
# # Tokenizes the input sequence upto 1024 characters.
# inputs = tokenizer([article], max_length=1024, return_tensors='tf',truncation = True)

# # Generate Summary
# summary_ids = model.generate(inputs['input_ids'])
# print([tokenizer.decode(g, clean_up_tokenization_spaces=False) for g in summary_ids])